# Zero-shot

In [3]:
from transformers import pipeline

# 1. INICIALIZACIÓN
# "zero-shot-classification" es la tarea mágica.
# El modelo se descarga solo la primera vez.
classifier = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-3")

# 2. TUS DATOS
# Texto del Cliente (Ej: Bran Stark)
texto_cliente = "Busca un lloc tranquil, silenciós i amb la millor fibra òptica per poder treballar sense límits."

# Tus "Temáticas Maestras" (Estas son las que agrupan tus 60 columnas de datos)
mis_temas = [
    "tranquilidad y silencio",
    "tecnología e internet",
    "vida nocturna y fiesta",
    "lujo y seguridad",
    "naturaleza y aire libre"
]

# 3. EJECUCIÓN
# Importante: multi_label=True permite que el texto pertenezca a varias categorías a la vez
# (Ej: Un barrio puede ser "Rico" Y "Seguro").
resultado = classifier(texto_cliente, mis_temas, multi_label=True)

# 4. VER RESULTADOS
print(f"Análisis para: '{texto_cliente}'\n")
for label, score in zip(resultado['labels'], resultado['scores']):
    print(f"- {label}: {score:.4f}")

Device set to use cpu


Análisis para: 'Busca un lloc tranquil, silenciós i amb la millor fibra òptica per poder treballar sense límits.'

- tranquilidad y silencio: 0.9965
- lujo y seguridad: 0.9491
- tecnología e internet: 0.4070
- naturaleza y aire libre: 0.0977
- vida nocturna y fiesta: 0.0200


In [7]:
import pandas as pd
from transformers import pipeline

# --- PASO 1: TU "DICCIONARIO MAESTRO" (HARDCODED) ---
# Las claves de este diccionario serán las opciones que le daremos a la IA.
TAXONOMIA = {
    "tranquilidad y silencio": ["ruido_db_inverso", "trafico_bajo"],
    "vida nocturna y fiesta":  ["densidad_bares", "num_restaurantes", "clubs"],
    "lujo y exclusividad y seguridad":     ["precio_vivienda", "renta_per_capita", "seguridad"],
    "naturaleza y aire libre": ["metros_parques", "arboles_calle", "calidad_aire"],
    "tecnología y conexión":   ["velocidad_fibra", "coworkings", "transporte_rapido"]
}

# --- PASO 2: INICIALIZAR EL TRANSFORMER ---
# Usamos las KEYS del diccionario como LABELS para el modelo
etiquetas_ia = list(TAXONOMIA.keys())
classifier = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-3")

def motor_de_recomendacion(texto_usuario, df_datos, umbral=0.25):
    """
    Conecta Texto -> IA -> Columnas -> Ranking
    """
    print(f" Analizando perfil: '{texto_usuario[:40]}...'")

    # A. LA IA PIENSA (Zero-Shot)
    # Le preguntamos: "¿A cuál de mis claves de diccionario se parece este texto?"
    resultado_ia = classifier(texto_usuario, etiquetas_ia, multi_label=True)
    print(resultado_ia)

    # B. PREPARAMOS EL CÁLCULO
    # Creamos una columna de score final inicializada a 0
    df_datos['score_final'] = 0.0

    # Lista para guardar las razones del match (para el frontend)
    justificaciones = []

    # C. EL PUENTE (Iteramos sobre los resultados de la IA)
    for concepto, score in zip(resultado_ia['labels'], resultado_ia['scores']):

        # Regla de Negocio: Si la IA no está segura (< umbral), ignoramos el concepto.
        if score < umbral:
            continue

        # Si pasa el filtro, buscamos qué columnas técnicas corresponden
        columnas_tecnicas = TAXONOMIA[concepto]

        # Guardamos la justificación textual
        justificaciones.append(f"{concepto.capitalize()} ({int(score*100)}%)")

        # D. CÁLCULO MATEMÁTICO VECTORIZADO (Pandas)
        # Sumamos al score final: (Valor de la columna en el barrio) * (Peso que dio la IA)
        for col in columnas_tecnicas:
            if col in df_datos.columns:
                # Aquí está la magia: Ponderación dinámica
                df_datos['score_final'] += df_datos[col] * score
            else:
                print(f"⚠️ Alerta: La columna '{col}' no existe en el DataFrame.")
    print(df_datos)

    # E. RESULTADO FINAL
    # Ordenamos los barrios de mayor a menor puntuación
    top_barrio = df_datos.sort_values('score_final', ascending=False).iloc[0]

    return top_barrio, justificaciones

# --- EJEMPLO DE USO ---

# 1. Simulemos datos de barrios (Normalizados 0-1)
datos = {
    'nombre': ['Downtown', 'Beverly Hills', 'Silver Lake'],
    'densidad_bares': [0.9, 0.1, 0.7],
    'ruido_db_inverso': [0.1, 0.9, 0.5],   # 1.0 es mucho silencio
    'precio_vivienda': [0.6, 1.0, 0.5],
    'metros_parques': [0.1, 0.8, 0.6],
    # ... (resto de columnas a 0 para el ejemplo)
    'trafico_bajo': [0.1, 0.8, 0.5],
    'num_restaurantes': [0.9, 0.5, 0.8],
    'clubs': [0.8, 0.0, 0.4],
    'renta_per_capita': [0.5, 1.0, 0.6],
    'seguridad_privada': [0.2, 1.0, 0.0],
    'arboles_calle': [0.1, 0.9, 0.7],
    'calidad_aire': [0.2, 0.8, 0.6],
    'velocidad_fibra': [0.9, 0.8, 0.7],
    'coworkings': [0.9, 0.2, 0.5],
    'transporte_rapido': [1.0, 0.1, 0.6]
}
df = pd.DataFrame(datos)

# 2. Caso: Bran Stark (Quiere silencio)
texto_bran = "Busca un lloc tranquil, silenciós i amb la millor fibra òptica."

# 3. Ejecutar Motor
ganador, razones = motor_de_recomendacion(texto_bran, df)

# 4. Mostrar Justificación (Requisito clave del PDF)
print("\n--- RECOMENDACIÓN ---")
print(f"Barrio Ganador: {ganador['nombre']}")
print(f"Score: {ganador['score_final']:.2f}")
print(f"Motivos principales: Se detectó interés en {', '.join(razones)}.")

Device set to use cpu


 Analizando perfil: 'Busca un lloc tranquil, silenciós i amb ...'
{'sequence': 'Busca un lloc tranquil, silenciós i amb la millor fibra òptica.', 'labels': ['tranquilidad y silencio', 'lujo y exclusividad', 'naturaleza y aire libre', 'tecnología y conexión', 'vida nocturna y fiesta'], 'scores': [0.9971404075622559, 0.8049415946006775, 0.3700588047504425, 0.35390347242355347, 0.019292250275611877]}
          nombre  densidad_bares  ruido_db_inverso  precio_vivienda  \
0       Downtown             0.9               0.1              0.6   
1  Beverly Hills             0.1               0.9              1.0   
2    Silver Lake             0.7               0.5              0.5   

   metros_parques  trafico_bajo  num_restaurantes  clubs  renta_per_capita  \
0             0.1           0.1               0.9    0.8               0.5   
1             0.8           0.8               0.5    0.0               1.0   
2             0.6           0.5               0.8    0.4               0.6   

 

In [ ]:
# DICCIONARIO MAESTRO (TAXONOMÍA)
# Clave (Izquierda): Lo que entiende la IA (Texto Humano)
# Valor (Derecha): La lista de columnas técnicas que activan ese concepto

TAXONOMIA = {
    # ---------------------------------------------------------
    # CONCEPTO 1: VIDA SOCIAL Y FIESTA
    # Ideal para: Tyrion Lannister [cite: 30, 31]
    # Qué busca la IA: Palabras como "fiesta", "social", "bares", "noche".
    # ---------------------------------------------------------
    "vida nocturna y social": [
        "osm_bares_count",         # Cantidad de bares
        "osm_restaurantes_count",  # Cantidad de restaurantes
        "osm_discotecas_count",    # 'club' o 'nightclub' en OSM
        "walkability_score"        # Caminabilidad (Tyrion no quiere taxis) [cite: 31]
    ],

    # ---------------------------------------------------------
    # CONCEPTO 2: TRANQUILIDAD Y SEGURIDAD
    # Ideal para: Cersei (Seguridad) [cite: 20] y Bran (Silencio) [cite: 23]
    # Qué busca la IA: "seguro", "silencio", "tranquilo", "privado".
    # ---------------------------------------------------------
    "seguridad y tranquilidad": [
        "crimen_inverso",          # IMPORTANTE: (1 - tasa_crimen). Alto valor = Seguro.
        "ruido_inverso",           # IMPORTANTE: (1 - nivel_ruido). Alto valor = Silencioso.
        "densidad_poblacion_inv"   # Poca gente = Más tranquilidad (Opcional)
    ],

    # ---------------------------------------------------------
    # CONCEPTO 3: NATURALEZA Y AIRE LIBRE
    # Ideal para: Jon Snow (Naturaleza) [cite: 25] y Daenerys (Perros) [cite: 18]
    # Qué busca la IA: "parques", "aire libre", "árboles", "gossos".
    # ---------------------------------------------------------
    "naturaleza y aire libre": [
        "osm_parques_area",        # Metros cuadrados de parque
        "osm_dog_park_count",      # Parques de perros (Clave para Daenerys)
        "calidad_aire_index",      # Si consigues datos ambientales
        "distancia_costa_inv"      # Cerca del mar (Opcional)
    ],

    # ---------------------------------------------------------
    # CONCEPTO 4: LUJO Y ESTATUS
    # Ideal para: Cersei [cite: 20]
    # Qué busca la IA: "lujo", "caro", "exclusivo", "élite".
    # ---------------------------------------------------------
    "lujo y exclusividad": [
        "precio_vivienda_avg",     # Precio medio alto
        "renta_per_capita",        # Ingresos altos (Census)
        "osm_tiendas_ropa",        # 'shop=clothes' (boutiques)
        "osm_escuelas_privadas"    # Colegios privados
    ],

    # ---------------------------------------------------------
    # CONCEPTO 5: VIDA PRÁCTICA Y TECNOLOGÍA
    # Ideal para: Arya (Transporte) [cite: 28] y Bran (Fibra) [cite: 23]
    # Qué busca la IA: "transporte", "conexión", "internet", "trabajo".
    # ---------------------------------------------------------
    "conectividad y servicios": [
        "osm_transporte_stops",    # Paradas de Bus/Metro
        "osm_coworking_count",     # Oficinas compartidas
        "velocidad_internet_avg",  # Dato de fibra (si lo encuentras) o proxys
        "cercania_supermercado"    # Vida práctica (Jon Snow) [cite: 25]
    ]
}